In [ ]:
# imports 

from src import sem_functions, city_functions, expert_functions
import importlib

importlib.reload(sem_functions)

from src.sem_functions import *
from src.city_functions import *
from src.expert_functions import *
import numpy as np
import pandas as pd
import seaborn as sns
import pickle
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
# import csv 

cities_one_hot_16 = pd.read_csv('/Users/katharina/Documents/PhD/Scripts_Paper_1/label_uncertainty_so2sat/data/cities_one_hot_16.csv').to_numpy()
cities_one_hot_named_16 = pd.read_csv('/Users/katharina/Documents/PhD/Scripts_Paper_1/label_uncertainty_so2sat/data/cities_one_hot_named_16.csv')
all_cities_16 = pd.read_csv('/Users/katharina/Documents/PhD/Scripts_Paper_1/label_uncertainty_so2sat/data/all_cities_16.csv').to_numpy()

## 1. Outer Bootstrap Loops for Expert Heterogeneity      

### 1.1 Computations

In [ ]:
# run B=3 outer bootstrap loops for full data and expert-specific data

In [ ]:
# "full"

list_indices_bootstrap = []
list_sem_16_bootstrap = []

B=3
b=0

while b<B:
    print('bootstrap iteration: ', b)

    N = len(cities_one_hot_16)
    ind_B = np.random.choice(N, size=N, replace=True)
    cities_one_hot_16_B = cities_one_hot_16[ind_B,:]
    sem_16_bootstrap = sem_fit_var(cities_one_hot_16_B, K=16, max_iter=20, rtol=1e-3, restarts=5)

    list_indices_bootstrap.append(ind_B)
    list_sem_16_bootstrap.append(sem_16_bootstrap)

    b=b+1

bootstrap iteration:  0
iteration 0
Loss: -549891.702612
Loss: -549366.684947
Loss: -549329.212762
better loss on iteration 0: -549329.2127615635
iteration 1
Loss: -549951.274636
Loss: -549378.050579
Loss: -549308.370258
better loss on iteration 1: -549308.3702582411
iteration 2
Loss: -549881.663875
Loss: -549553.823622
Loss: -549331.818396
iteration 3
Loss: -549915.561732
Loss: -549327.295745
Loss: -549376.631943
iteration 4


In [1]:
# save results
pi_bootstrap=[]
theta_bootstrap=[]
tau_bootstrap_org = []
tau_bootstrap=[]
b=0

for sem in list_sem_16_bootstrap:
    pi_bootstrap.append(sem[1])
    theta_bootstrap.append(sem[2])
    tau_bootstrap_org.append(sem[4])
    ind_B = list_indices_bootstrap[b]
    tau_bootstrap.append(e_step_nozerocorrection(cities_one_hot_16[ind_B,:], sem[1], sem[2]))
    b=b+1

NameError: name 'list_sem_16_bootstrap' is not defined

In [ ]:
# "experts"

one_hot_experts = expert_data(all_cities_16, cities_one_hot_16)
list_sem_16_experts_bootstrap = []

for b in range(len(list_sem_16_bootstrap)):

    print('bootstrap iteration: ', b)
    ind_B = list_indices_bootstrap[b]

    one_hot_experts_bootstrap = []
    for data in one_hot_experts:
        one_hot_experts_bootstrap.append(data[ind_B])

    sem_16_experts = expert_sem(one_hot_experts_bootstrap, K=16, rtol=1e-3, max_iter=50, restarts=5)

    list_sem_16_experts_bootstrap.append(sem_16_experts)

In [ ]:
# save results 
list_pi_b_experts=[]
list_theta_b_experts=[]
list_tau_b_experts_org = []
list_tau_b_experts=[]

b=0
for sem_boot in list_sem_16_experts_bootstrap:

    ind_B = list_indices_bootstrap[b]
    pi_b_experts=[]
    theta_b_experts=[]
    tau_b_experts_org = []
    tau_b_experts=[]

    j=0
    for sem in sem_boot:
        pi_b_experts.append(sem[1])
        theta_b_experts.append(sem[2])
        tau_b_experts_org.append(sem[4])
        tau_b_experts.append(e_step_nozerocorrection(one_hot_experts[j][ind_B,:], sem[1], sem[2]))
        j=j+1

    list_pi_b_experts.append(pi_b_experts)
    list_theta_b_experts.append(theta_b_experts)
    list_tau_b_experts_org.append(tau_b_experts_org)
    list_tau_b_experts.append(tau_b_experts)

    b=b+1

### 1.2 Results 

In [ ]:
### Outer Bootstrap: 0

In [ ]:
outer_b=0
bootstrap_sem = list_sem_16_experts_bootstrap[outer_b]
bootstrap_tau_experts = list_tau_b_experts[outer_b]
outer_bootstrap_ind = list_indices_bootstrap[outer_b]

In [ ]:
# original stats for all experts j 

K=16

log_LH_list_B0 = []

for j in range(11):
    print('expert ', j)
    tau_j = bootstrap_tau_experts[j]
    e_votes = all_cities_16[outer_bootstrap_ind,j]

    log_LH_j_i = [np.sum([0 if tau_j[i,k] == 0
                          else 1*np.log(tau_j[i,k]) if e_votes[i]==k
    else 0*np.log(tau_j[i,k]) for k in range(K)]) for i in range(len(e_votes))]

    log_LH_list_B0.append(np.sum(log_LH_j_i))

In [ ]:
# bootstrapping images while keeping tau fixed

log_LH_boot_inner_list_B0 = []
B_inner = 50

for b in range(B_inner):
    print('bootstrap: ', b)

    N = len(all_cities_16)
    ind_B_inner = np.random.choice(N, size=N, replace=True)

    log_LH_b_inner = []

    for j in range(11):
        print('expert ', j)
        tau_j = bootstrap_tau_experts[j][ind_B_inner]
        e_votes = all_cities_16[outer_bootstrap_ind[ind_B_inner],j]

        log_LH_j_i = [np.sum([0 if tau_j[i,k] == 0
                              else 1*np.log(tau_j[i,k]) if e_votes[i]==k
        else 0*np.log(tau_j[i,k]) for k in range(K)]) for i in range(len(e_votes))]

        log_LH_b_inner.append(np.sum(log_LH_j_i))

    log_LH_boot_inner_list_B0.append(log_LH_b_inner)

In [ ]:
# extract arrays of original and bootstrapped values 
expert_bootstrapped_vals_B0 = []

for j in range(11):
    vals = []
    for b in log_LH_boot_inner_list_B0:
        vals.append(-b[j])
    expert_bootstrapped_vals_B0.append(vals)

In [ ]:
# look at pairwise differences 
d_j_list_B0 = []
for j in range(11):
    x_j = -log_LH_list_B0[j]
    sum_j = (-np.sum(log_LH_list_B0) - x_j)/10
    d_j = np.abs(x_j - sum_j)
    d_j_list_B0.append(d_j)

d_j_list_bootstrapped_B0 = []
for log_LH in log_LH_boot_inner_list_B0:

    d_j_list_b = []
    for j in range(11):
        x_j_b = -log_LH[j]
        sum_j_b = (-np.sum(log_LH) - x_j_b)/10
        d_j_b = np.abs(x_j_b - sum_j_b)
        d_j_list_b.append(d_j_b)

    d_j_list_bootstrapped_B0.append(d_j_list_b)

# extract arrays of original and bootstrapped values 

expert_bootstrapped_distances_B0 = []

for j in range(11):
    d_vals = []
    for d in d_j_list_bootstrapped_B0:
        d_vals.append(d[j])
    expert_bootstrapped_distances_B0.append(d_vals)

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(12, 8), dpi=80)
for j in range(len(expert_bootstrapped_distances_B0)):
    sns.set_style('whitegrid')
    sns.kdeplot(np.array(expert_bootstrapped_distances_B0[j]), bw=0.5,label=j+1)
plt.legend()
plt.show()

In [ ]:
### Outer Bootstrap 1+2

# repeat above 

## 2. Outer Bootstrap Loop for City Differences 